In [1]:
import io, os, importlib, pickle
import datetime
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
from ir_crosslingual.utils import paths
importlib.reload(paths)

from ir_crosslingual.sentences import sentences
importlib.reload(sentences)

from ir_crosslingual.supervised_classification import sup_model
importlib.reload(sup_model)

<module 'ir_crosslingual.supervised_classification.sup_model' from '/Users/i500969/Desktop/Admin/Uni-Mannheim/02_Courses/2020_FSS/Information-Retrieval/03_Project/03_Implementation/05_Master/ir-crosslingual/ir_crosslingual/supervised_classification/sup_model.py'>

In [15]:
def time(start, stop, message):
    print('Computation time {}: {}'.format(message, stop-start))
    print('Finished at: {}'.format(datetime.datetime.now()))

## Load models

In [17]:
lr_model, lr_prepared_features, lr_features_dict = sup_model.SupModel.load_model(name='logReg_v0.2')
lr_features = [feature for values in lr_features_dict.values() for feature in values]

In [18]:
mlp_model, mlp_prepared_features, mlp_features_dict = sup_model.SupModel.load_model(name='mlp_base')
mlp_features = [feature for values in mlp_features_dict.values() for feature in values]

## English/French

In [4]:
enfr_sens, enfr_chunks = sentences.Sentences.load_chunks_from_file('en', 'fr')

---- INFO: Learn projection matrix for en-fr
---- INFO: Found 10369 valid translation pairs in expert dictionary.
---- INFO: 503 other pairs contained at least one unknown word (0 in source language, 503 in target language).
---- DONE: Seed dictionary extracted for the languages: en-fr
---- INFO: Resulting subspace dimension: (10369, 300)
---- INFO: Resulting subspace dimension: (10369, 300)
---- DONE: Projection matrix learned from en to fr
---- INFO: Learn projection matrix for fr-en
---- INFO: Found 7938 valid translation pairs in expert dictionary.
---- INFO: 332 other pairs contained at least one unknown word (0 in source language, 332 in target language).
---- DONE: Seed dictionary extracted for the languages: fr-en
---- INFO: Resulting subspace dimension: (7938, 300)
---- INFO: Resulting subspace dimension: (7938, 300)
---- DONE: Projection matrix learned from fr to en
---- DONE: All chunks loaded


In [ ]:
# Extract vector elements

In [10]:
# Apply scaling to chunks

In [13]:
enfr_sens.test_collection = pd.concat(enfr_chunks)

In [16]:
# Evaluation of base logistic regression
# print('Evaluation on logistic regression model')
start = datetime.datetime.now()
sup = sup_model.SupModel()
sup.evaluate_boolean(lr_model, enfr_sens, lr_features)
print('Accuracy: {}'.format(sup.accuracy))
print('Precision: {}'.format(sup.precision))
print('Recall: {}'.format(sup.recall))
print('F1: {}'.format(sup.f1))
stop = datetime.datetime.now()
time(start, stop, 'evaluating boolean')
start = datetime.datetime.now()
print('MAP: {}'.format(sup.compute_map(lr_model, enfr_sens, lr_features)))
stop = datetime.datetime.now()
time(start, stop, 'computing the MAP score')

Accuracy: 0.8416989
Precision: 0.0005562718546646647
Recall: 0.8679802955665025
F1: 0.0011118311581532498
Computation time evaluating boolean: 0:00:25.422572
Finished at: 2020-05-18 02:49:27.610022
---- INFO: Probabilities predicted
---- INFO: Dataframe with evaluation ranking created
---- INFO: Probabilities sorted for each query
---- INFO: Index of ranking of true translation retrieved
MAP: 0.29103383172440733
Computation time computing the MAP score: 0:13:19.946869
Finished at: 2020-05-18 03:02:47.557105


In [22]:
print('Evaluation on baseline MLP model')
print('-'*60)
start = datetime.datetime.now()
sup = sup_model.SupModel()
sup.evaluate_boolean(mlp_model, enfr_sens, mlp_features)
print('Accuracy: {}'.format(sup.accuracy))
print('Precision: {}'.format(sup.precision))
print('Recall: {}'.format(sup.recall))
print('F1: {}'.format(sup.f1))
stop = datetime.datetime.now()
time(start, stop, 'evaluating boolean')
start = datetime.datetime.now()
print('MAP: {}'.format(sup.compute_map(mlp_model, enfr_sens, mlp_features)))
stop = datetime.datetime.now()
time(start, stop, 'computing the MAP score')

Evaluation on baseline MLP model
------------------------------------------------------------
Accuracy: 0.2951756
Precision: 4.5261921089033034e-05
Recall: 0.3142857142857143
F1: 9.051080724574478e-05
Computation time evaluating boolean: 0:00:31.599639
Finished at: 2020-05-18 03:06:38.780502
---- INFO: Probabilities predicted
---- INFO: Dataframe with evaluation ranking created
---- INFO: Probabilities sorted for each query
---- INFO: Index of ranking of true translation retrieved
MAP: 0.0003518459551696094
Computation time computing the MAP score: 0:13:42.196935
Finished at: 2020-05-18 03:20:20.977603
